### Objective
1. What is recommender Systems
2. Types of recommender Systems
3. Using basic recommender Systems from mlxtend for online retail data 

<hr>

### 1. Recommender System
* This system suggest user to buy, connect, sell, watch.
* Lot of distribution channel makes money using it.

### 2. Types of recommender Systems
* Content Based Filtering - Recommendations are based on metadata collected from a user's history & interaction. Based on your previous watched movies, system will recommend you new movies. Used earlier days. Not much in production.

* Collaborative Filtering - Collecting information from interactions from many other uses to derive suggestions for you.This is what is used in production

* Hybrid Filtering - Combining both content based filtering & collaborative filtering

In [1]:
import pandas as pd

In [3]:
articles_df = pd.read_csv('Data/2234_3774_bundle_archive/shared_articles.csv')

In [6]:
articles_df = articles_df[articles_df.eventType == 'CONTENT SHARED']

In [7]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3047 entries, 1 to 3121
Data columns (total 13 columns):
timestamp          3047 non-null int64
eventType          3047 non-null object
contentId          3047 non-null int64
authorPersonId     3047 non-null int64
authorSessionId    3047 non-null int64
authorUserAgent    669 non-null object
authorRegion       669 non-null object
authorCountry      669 non-null object
contentType        3047 non-null object
url                3047 non-null object
title              3047 non-null object
text               3047 non-null object
lang               3047 non-null object
dtypes: int64(4), object(9)
memory usage: 333.3+ KB


In [13]:
articles_df.sample(10)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
854,1462799402,CONTENT SHARED,-6369211973034956889,3302556033962996625,-1309754598637396315,NaN,NaN,NaN,HTML,http://www.androiddesignpatterns.com/2014/12/a...,Getting Started with Activity & Fragment Trans...,This post gives a brief overview of Transition...,en
141,1459518806,CONTENT SHARED,6090517797383271316,-1387464358334758758,4032304591654000532,NaN,NaN,NaN,HTML,http://googlediscovery.com/2016/04/01/google-a...,"April Fools - Google apresenta o realBooks, a ...","Livros digitais são ótimos, mas todos sabemos ...",pt
1207,1464871461,CONTENT SHARED,-7255737497176527784,3609194402293569455,6147896666617046192,NaN,NaN,NaN,HTML,http://epocanegocios.globo.com/Empresa/noticia...,Siga o líder,Aldemir Bendine (Petrobras) (Foto: Agência O G...,pt
2665,1477572325,CONTENT SHARED,-5011709367955107239,-1251984896177895077,-3411758262273298934,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,http://jcrs.uol.com.br/_conteudo/2016/10/econo...,"Vale reverte prejuízo e tem lucro de R$ 1,8 bi...",Investimentos da mineradora no período caíram ...,pt
2613,1476984097,CONTENT SHARED,8289311442635691404,3396828831632510915,1233659230241778611,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,GA,US,HTML,http://news.microsoft.com/features/have-a-coke...,"Have a Coke, take a photo and celebrate the Co...","A few months ago, the Microsoft team behind Ho...",en
1617,1467391602,CONTENT SHARED,7434270606888620096,3891637997717104548,-2108423950476348958,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/06/S...,Stackdriver Debugger now works with source cod...,Stackdriver Debugger has always worked with so...,en
2073,1471316974,CONTENT SHARED,6638782839540370044,-1032019229384696495,880025729283447629,NaN,NaN,NaN,HTML,https://www.datanami.com/2016/08/15/python-eat...,Python Eats Into R as SAS Dominance Fades,A new survey of data science tools shows that ...,en
290,1460122006,CONTENT SHARED,-5578229678262992168,-6316613156648676087,5673696108944885080,NaN,NaN,NaN,HTML,https://www.yahoo.com/tech/shut-up-don-t-take-...,Don't take my money: Why mobile payments haven...,A buck says you won't buy your next tank of ga...,en
133,1459467851,CONTENT SHARED,-8349788035147634450,-1032019229384696495,-4690277498179936282,NaN,NaN,NaN,HTML,http://uxmag.com/articles/how-mature-is-your-o...,How Mature is Your Organization when it Comes ...,"Every organization has its own goals, processe...",en
1299,1465335780,CONTENT SHARED,3547081132182529111,-1032019229384696495,-7696592431575292648,NaN,NaN,NaN,HTML,http://www.grubstreet.com/bestofnewyork/best-c...,The Absolute Best Coffee in New York,"Photo: Melissa Hom For six weeks, the editors ...",en


In [8]:
interaction_df = pd.read_csv('Data/2234_3774_bundle_archive/users_interactions.csv')

In [10]:
interaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72312 entries, 0 to 72311
Data columns (total 8 columns):
timestamp      72312 non-null int64
eventType      72312 non-null object
contentId      72312 non-null int64
personId       72312 non-null int64
sessionId      72312 non-null int64
userAgent      56918 non-null object
userRegion     56907 non-null object
userCountry    56918 non-null object
dtypes: int64(4), object(4)
memory usage: 4.4+ MB


In [11]:
interaction_df.sample(10)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
57572,1475855626,VIEW,5854206600849997966,4209517478660372522,2815235160052144552,Android - Native Mobile App,SP,BR
12076,1464656389,VIEW,3998743085432844199,-1032019229384696495,-484145324341325773,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
31345,1467136594,VIEW,7270966256391553686,-89388927330481219,-8601760165379048963,NaN,NaN,NaN
24545,1466004015,VIEW,-3581194288660477595,-5409830449698398534,-3487889034139626978,NaN,NaN,NaN
46127,1472158850,LIKE,-2271000159302779222,-709287718034731589,-1147236519157684050,NaN,NaN,NaN
67211,1484654121,VIEW,-6348943063155744346,-3596626804281480007,-2426518688510377485,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1...,SP,BR
5997,1465419297,LIKE,943818026930898372,-2337468721613240548,-4003972290196927412,NaN,NaN,NaN
6543,1460659529,VIEW,9207286802575546269,-1032019229384696495,1822267539745436764,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,NY,US
34441,1469099805,VIEW,4259370161044254504,3302556033962996625,8855044036409885533,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
21197,1466706515,VIEW,6997620589258672675,8368708280460693260,-2291023684625068468,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5...,SP,BR


In [12]:
interaction_df.eventType.unique()

array(['VIEW', 'FOLLOW', 'BOOKMARK', 'LIKE', 'COMMENT CREATED'],
      dtype=object)

In [14]:
event_type_strength = {
    'VIEW': 1,
    'LIKE': 2,
    'BOOKMARK':3,
    'FOLLOW':4,
    'COMMENT CREATED':5
}

In [16]:
interaction_df['event_type_strength'] = interaction_df.eventType.map(event_type_strength)

* Recommender System classic problem is cold-start issue. If you join a new platform, the platform don't really know what to recommand
* Thus, we only want recommendation system to kick in after user has done some activity. So, we will consider users with more than 5 activity

In [19]:
user_interaction_count = interaction_df.groupby(['personId','contentId']).size().groupby('personId').size()

In [20]:
user_interaction_count

personId
-9223121837663643404    43
-9212075797126931087     5
-9207251133131336884     7
-9199575329909162940    11
-9196668942822132778     7
                        ..
 9165571805999894845     5
 9187866633451383747    17
 9191849144618614467    12
 9199170757466086545    14
 9210530975708218054    63
Length: 1895, dtype: int64

In [28]:
user_with_enough_interaction = user_interaction_count[user_interaction_count >= 5]

### Understanding from the above commands
* user_interaction_count tells how much interactions each user is doing
* user_with_enough_interaction is users with more than 5 interactions
* We are doing this to avoid cold start problems

In [35]:
user_with_enough_interaction_df = pd.DataFrame(user_with_enough_interaction).reset_index()[['personId']]

In [29]:
interaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72312 entries, 0 to 72311
Data columns (total 9 columns):
timestamp              72312 non-null int64
eventType              72312 non-null object
contentId              72312 non-null int64
personId               72312 non-null int64
sessionId              72312 non-null int64
userAgent              56918 non-null object
userRegion             56907 non-null object
userCountry            56918 non-null object
event_type_strength    72312 non-null int64
dtypes: int64(5), object(4)
memory usage: 5.0+ MB


In [38]:
interactions_from_chosen_users = interaction_df.merge(user_with_enough_interaction_df, how='right', left_on='personId', right_on='personId')

In [39]:
interactions_from_chosen_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69868 entries, 0 to 69867
Data columns (total 9 columns):
timestamp              69868 non-null int64
eventType              69868 non-null object
contentId              69868 non-null int64
personId               69868 non-null int64
sessionId              69868 non-null int64
userAgent              54877 non-null object
userRegion             54867 non-null object
userCountry            54877 non-null object
event_type_strength    69868 non-null int64
dtypes: int64(5), object(4)
memory usage: 5.3+ MB


In [43]:
interaction_full_df = pd.DataFrame(interactions_from_chosen_users.groupby(['personId','contentId'])['event_type_strength'].sum().reset_index())

In [48]:
df = interaction_full_df[interaction_full_df.personId == -9223121837663643404]

In [54]:
df.sort_values(by='event_type_strength', ascending=False)[:5]

,personId,contentId,event_type_strength
4,-9223121837663643404,-7423191370472335463,8
40,-9223121837663643404,8302949268716967655,4
22,-9223121837663643404,-447851796385928420,3
21,-9223121837663643404,-559964548932224920,2
10,-9223121837663643404,-6545872007932025533,2


In [55]:
### Based on this data we will do content based filtering

In [56]:
# We will use cosine similarity here
class ContentBasedRecommenderSystem:
    pass

In [57]:
class CollaborativeRecommenderSystem:
    pass